In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import OP
from scipy.optimize import fsolve

## 1. 求解区间

In [13]:
def solve_interval(S0, r, sigma, T, steps, I, n, m, type = 'accumulator'):
    ##
    MC = OP.MonteCarlo()
    MC.simulations(S0=S0, r=r, sigma=sigma, T=T, steps=steps, I=I)
    def func(interval):
        if type == 'accumulator':
            KO = S0 + interval/2
            S = S0 - interval/2
            y = MC.accumulator(KO=KO, r=r, S=S, I=I, n=n, m=m)
        else:
            KO = S0 - interval/2
            S = S0 + interval/2
            y = MC.decumulator(KO=KO, r=r, S=S, I=I, n=n, m=m)
        return y
    interval = fsolve(func, [500])
    return interval

In [16]:
S0 = 5658
S = 5742
# KO_1 = 5558
KO_2 = 5558
r = 0.05
T = 30/252
sigma = 0.115
steps = 30
n = 1/steps
m = 3
I = 10000

interval = solve_interval(S0, r, sigma, T, steps, I, n, m, type = 'decumulator')
interval

array([282.1879482])

### 验证

In [17]:
S = S0 + interval/2
KO_2 = S0 - interval/2

MC = OP.MonteCarlo()
MC.simulations(S0=S0, r=r, sigma=sigma, T=T, steps=steps, I=I)

decumulator_price = MC.decumulator(S=S, KO=KO_2, r=r, I=I, n=n, m=m)
print(decumulator_price)


4.431642788165302


## 2. 计算波动率

In [18]:
def find_volatility(data, type='annually'):
    daily_volatility = np.std(data, ddof=1)
    annualized_volatility = daily_volatility * np.sqrt(252)
    if type =='daily':
        return daily_volatility
    elif type =='annually':
        return annualized_volatility
    else:
        raise ValueError('type not daily or annually')

## 3. 计算delta

In [35]:
def find_delta(S0, r, T, sigma, I, S, KO, n, m):
    S0_1 = S0+0.005*S0
    S0_2 = S0-0.005*S0
    MC_1 = OP.MonteCarlo()
    MC_2 = OP.MonteCarlo()
    steps = int(T*252)

    MC_1.simulations(S0_1,r,T,sigma,I,steps)
    MC_2.simulations(S0_2,r,T,sigma,I,steps)

    pv_1 = MC_1.accumulator(S=S, KO=KO, r=r, I=I, n=n, m=m)
    pv_2 = MC_1.accumulator(S=S, KO=KO, r=r, I=I, n=n, m=m)

    delta = (pv_1-pv_2)/(0.01*S0*n*steps) #Q：对于累计/沽期权，标的资产应是什么？
    return delta

## 4. 回测模拟

In [44]:
def back_testing(path, r, sigma, I, n, m):
    #TO DO
    S0 = path[0]
    T = (len(path)-1)/252
    steps = int(T*252)
    interval = solve_interval(S0, r, sigma, T, steps, I, n, m, type = 'accumulator')
    S = S0 - interval/2
    KO = S0 + interval/2

    delta = find_delta(S0, r, T, sigma, I, S, KO, n, m)
    hold = delta*n*steps #对冲持有的点数，如持有100吨石油

    fee = 0
    contract = 0
    hedge = 0
    for i in range(1, len(path)-1):
        T_ = (len(path)-i-1)/252
        S0_ = path[i]
        steps_ = T_*252
        #计算每天期权合约赔付
        if S0_ >= KO:
            contract += 0 
        elif S0_ < KO and S0_ > S:
            contract += (S0_-S)*n
        else:
            contract += (S0_-S)*m*n
        #计算每天对冲端收益
        hedge += hold*(S0_-path[i-1])

        #计算新的delta并得出新的对冲端持仓
        delta = find_delta(S0_, r, T_, sigma, I, S, KO, n, m)
        hold = delta*n*steps_

    #最后一天不需要计算delta和hold，计算收益即可
    if path[-1] >= KO:
        contract += 0 
    elif path[-1] < KO and S0_ > S:
        contract += (path[-1]-S)*n
    else:
        contract += (path[-1]-S)*m*n
    
    hedge += hold*(path[-1]-path[-2])


    payoff = fee-contract+hedge
    return payoff

## 5. 多时期回测大循环

读取数据

In [45]:
data = pd.read_excel('gold_price.xlsx')
price_path = data['gold_price']
price_path = price_path.to_numpy()
price_path = price_path[-90:]
print(len(price_path))

90


In [46]:
payoff = np.zeros(int((len(price_path)-30)/30))

r = 0.03
I = 10000
n = 10
m = 3
index = 0
for i in range(30, len(price_path)-30, 30):
    sigma = find_volatility(price_path[i-30: i-1], type='annually')
    payoff[index] =  back_testing(price_path[i:i+30], r, sigma, I, n, m)
    index += 1

payoff[index] = back_testing(price_path[-30:], r, sigma, I, n, m)


#计算平均收益率
avg_payoff = np.mean(payoff)

#计算胜率
win = 0
lose = 0
for i in payoff:
    if i >= 0:
        win += 1
    else:
        lose += 1
winning_rate = win/len(payoff)

#计算最大亏损
max_lose = payoff.min()

#计算最大收益

max_win = payoff.max()

print(f'平均收益率：{avg_payoff}')
print(f'胜率：{winning_rate}')
print(f'最大亏损：{max_lose}')
print(f'最大收益：{max_win}')
